In [3]:
pip install cohere

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 652.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 14.1 MB/s eta 0:00:00
  Attempting uninstall: importlib_metadata
    Found existing installation: importlib-metadata 7.0.1
    Uninstalling importlib-metadata-7.0.1:
      Successfully uninstalled importlib-metadata-7.0.1


In [4]:
import cohere
import uuid
co = cohere.Client('A5aqcw7FeLSL5sqynr0F2NMlZ8S7P9zQvFaeUJs0')

In [5]:
conversation_id = str(uuid.uuid4())

# print(conversation_id)

preamble_override = "You are an expert public speaking coach"

print("Starting the chat, type ""quit"" to end the chat")

while True:
    message = input("User: ")
    if message.lower == "quit":
        print("Chat ended")
        break

    response = co.chat(conversation_id=conversation_id,
                       message=message,
                       preamble_override=preamble_override,
                       stream=True,
                       chat_history=True)

    print("Chatbot: ", end='')
    print(response, end='')

    for event in response:
        if event.event_type == "text-generation":
            print(event.text, end='')

Starting the chat, type quit to end the chat
User: quit
Chatbot: cohere.StreamingChat {
	response: <Response [400]>
	texts: []
	response_id: None
	conversation_id: None
	generation_id: None
	preamble: None
	prompt: None
	chat_history: None
	finish_reason: None
	token_count: None
	meta: None
	is_search_required: None
	citations: None
	documents: None
	search_results: None
	search_queries: None
}

KeyboardInterrupt: Interrupted by user

In [6]:
%pip install --upgrade pip
%pip install --disable-pip-version-check \
    torch==1.13.1 \
    torchdata==0.5.1 --quiet

%pip install \
    transformers==4.27.2 \
    datasets==2.11.0 \
    evaluate==0.4.0 \
    rouge_score==0.1.2 \
    loralib==0.1.1 \
    peft==0.3.0 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.4 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 64.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu121 requires torch==2.1.0, but you have torch 1.13.1 which is incompatible.
torchtext 0.16.0 requires torch=

In [7]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer # Used for tranining models from Huggingface
import torch
import time
import evaluate
import pandas as pd
import numpy as np


In [8]:
# Experiment with Dialog dataset from Huggingface

huggingface_dataset_name = "knkarthick/dialogsum"

dataset = load_dataset(huggingface_dataset_name)

dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-cd36827d3490488d/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
})

In [9]:
model_name = 'google/flan-t5-base'

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)

tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [10]:
def print_number_of_trainable_model_parameters(model) :
  trainable_model_params = 0
  all_model_params = 0

  for _, param in model.named_parameters() : # _ means unused placeholder parameters
    all_model_params += param.numel()

    if param.requires_grad :
      trainable_model_params += param.numel()
  return f"""trainable model parameters : {trainable_model_params}
all model parameters : {all_model_params}
percentage model parameters : {100 * trainable_model_params / all_model_params:.2f}%"""

print (print_number_of_trainable_model_parameters(original_model))

trainable model parameters : 247577856
all model parameters : 247577856
percentage model parameters : 100.00%


In [11]:
index = 201
dialogue = dataset['test'][index]['dialogue']
summary = dataset['test'][index]['summary']

prompt = f"""
Summarize the following conversation

{dialogue}
Summary :
"""

inputs = tokenizer(prompt, return_tensors='pt')

output = tokenizer.decode(
    original_model.generate(
        inputs["input_ids"],
        max_new_tokens=200
    )[0],
    skip_special_tokens=True
)

dash_line = "_".join('' for x in range(100))

print(dash_line)
print(f'INPUT PROMPT : \n {prompt}')

print(dash_line)
print(f'BASELINE HUMAN SUMMARY : \n {summary} \n')

print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT : \n {output}')


___________________________________________________________________________________________________
INPUT PROMPT : 
 
Summarize the following conversation

#Person1#: Where to, miss?
#Person2#: Hi! Crenshaw and Hawthorne, at the Holiday Inn that is on that corner.
#Person1#: Sure thing. So, where are you flying in from?
#Person2#: From China.
#Person1#: Really? You don't look very Chinese to me, if you don't mind me saying so.
#Person2#: It's fine. I am actually from Mexico. I was in China on a business trip, visiting some local companies that manufacture bathroom products.
#Person1#: Wow sounds interesting! Excuse me if I am being a bit nosy but, how old are you?
#Person2#: Don't you know it's rude to ask a lady her age?
#Person1#: Don't get me wrong! It's just that you seem so young and already doing business overseas!
#Person2#: Well thank you! In that case, I am 26 years old, and what about yourself?
#Person1#: I am 40 years old and was born and raised here in the good old U. S of 

In [12]:
def tokenize_function(example) :
  start_prompt = 'Summarize the following conversation - UPDATED. \n\n'
  end_prompt = '\n\n Summary : '

  prompt = [start_prompt + dialogue + end_prompt for dialogue in example['dialogue']]

  example['input_ids'] = tokenizer(prompt, padding="max_length", truncation = True, return_tensors='pt').input_ids
  example['labels'] = tokenizer(example["summary"], padding="max_length", truncation=True, return_tensors='pt').input_ids

  return example

In [13]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['id', 'topic', 'dialogue', 'summary'])
tokenized_datasets = tokenized_datasets.filter(lambda example, index : index % 100 == 0, with_indices=True)

Map:   0%|          | 0/12460 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Filter:   0%|          | 0/12460 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1500 [00:00<?, ? examples/s]

Filter:   0%|          | 0/500 [00:00<?, ? examples/s]

In [14]:
print(f'Shapes of the datasets : ')
print(f"Training : {tokenized_datasets['train'].shape}")
print(f"Validation : {tokenized_datasets['validation'].shape}")
print(f"Test : {tokenized_datasets['test'].shape}")

print(tokenized_datasets)

Shapes of the datasets : 
Training : (125, 2)
Validation : (5, 2)
Test : (15, 2)
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 125
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 15
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 5
    })
})


In [17]:
output_dir = f'./dialogue-summary-training-{str(int(time.time()))}'

training_args = TrainingArguments(
    output_dir = output_dir,
    learning_rate = 1e-5,
    num_train_epochs=1,
    weight_decay = 0.01,
    logging_steps = 1,
    max_steps = 1
)

trainer = Trainer(
    model = original_model,
    args = training_args,
    train_dataset = tokenized_datasets['train'],
    eval_dataset = tokenized_datasets['validation']
)

trainer

In [ ]:
trainer.train() # might take 1.5 hours in Colab




In [49]:
instruct_model = AutoModelForSeq2SeqLM.from_pretrained("./dialogue-summary-training-1709291286/", torch_dtype = torch.bfloat16)


OSError: ./dialogue-summary-training-1709291286/ does not appear to have a file named config.json. Checkout 'https://huggingface.co/./dialogue-summary-training-1709291286//None' for available files.

In [1]:
!pip install replicate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.8 MB/s eta 0:00:00


In [2]:
from getpass import getpass
import os

# get token from https://replicate.com/account/api-tokens
REPLICATE_API_TOKEN = getpass()
os.environ["REPLICATE_API_TOKEN"] = REPLICATE_API_TOKEN

··········


In [3]:
import replicate

In [7]:
output = replicate.run(
  "stability-ai/sdxl:39ed52f2a78e934b3ba6e2a89f5b1c712de7dfea535525255b1aa35c5565e08b",
  input={"prompt": "an iguana on the beach, pointillism"}
)

# ['https://replicate.delivery/pbxt/VJyWBjIYgqqCCBEhpkCqdevTgAJbl4fg62aO4o9A0x85CgNSA/out-0.png']
from IPython.display import Image
Image(url=output[0])